In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['violent_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['violent_two_year'].values

In [3]:
#### CART
depth = [8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                          learning_rate=learning_rate, 
                          depth=depth, 
                          estimators=estimators, 
                          seed=816)

In [4]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.6355514659421205, 0.621221407809933)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['violent_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['violent_two_year'].values

## columns 
cols = KY_X.columns

In [9]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.003, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge19.0',
 'age_at_current_charge20.0',
 'age_at_current_charge21.0',
 'age_at_current_charge22.0',
 'age_at_current_charge23.0',
 'age_at_current_charge24.0',
 'age_at_current_charge27.0',
 'age_at_current_charge30.0',
 'age_at_current_charge31.0',
 'age_at_current_charge33.0',
 'age_at_current_charge34.0',
 'age_at_current_charge35.0',
 'age_at_current_charge37.0',
 'age_at_current_charge38.0',
 'age_at_current_charge40.0',
 'age_at_current_charge46.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_charges1',
 'p_violence1',
 'p_violence2',
 'p_violence3',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_misdeassult1',
 'p_traffic1',
 'p_drug1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'current_violence1']

In [10]:
c_grid={'C': [0.001, 0.002, 0.003]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [12]:
stumps_summary['best_params'], stumps_summary['FL_score'], stumps_summary['auc_diffs']

({'C': 0.003}, 0.6596771136105144, 0.0007569652469914256)

### RiskSLIM

In [13]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['violent_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['violent_two_year'].values

## columns 
cols = KY_X.columns

In [23]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 
                                        0.0008, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

20

In [24]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [25]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'violent_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime= 2000, 
                                max_offset = 100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 01:54 PM | 253 rows in lookup table
11/12/19 @ 01:54 PM | ------------------------------------------------------------
11/12/19 @ 01:54 PM | runnning initialization procedure
11/12/19 @ 01:54 PM | ------------------------------------------------------------
11/12/19 @ 01:54 PM | CPA produced 2 cuts
11/12/19 @ 01:54 PM | running naive rounding on 6 solutions
11/12/19 @ 01:54 PM | best objective value: 0.3657
11/12/19 @ 01:54 PM | rounding produced 5 integer solutions
11/12/19 @ 01:54 PM | best objective value is 0.3457
11/12/19 @ 01:54 PM | running sequential rounding on 6 solutions
11/12/19 @ 01:54 PM | best objective value: 0.3657
11/12/19 @ 01:54 PM | sequential rounding produced 3 integer solutions
11/12/19 @ 01:54 PM | best objective value: 0.3457
11/12/19 @ 01:54 PM | polishing 8 solutions
11/12/19 @ 01:54 PM | best objective value: 0.3457
11/12/19 @ 01:54 PM | polishing produced 5 integer solutions
11/12/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:54 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2206.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 539303 115349        cutoff              0.2143        0.2092  3315150    2.42%          rho_13 D 539303 539301     35
 552239 117003        cutoff              0.2143        0.2093  3386740    2.36%        alpha_16 U 552239 552236     38
 566158 120367        0.2143     8        0.2143        0.2094  3458557    2.33%           rho_5 D 566158 566156     32
 578114 120910        cutoff              0.2143        0.2095  3530666    2.27%        alpha_12 U 578114 578112     22
 590883 122155        cutoff              0.2143        0.2096  3602740    2.22%           rho_5 D 590883 590881     32
 604031 124591        0.2097    16        0.2143        0.2097  3668246    2.18%           rho_0 U 604031 604023     29
 617590 125969        0.2098    12        0.2143        0.2098  3735210    2.14%           rho_9 U 617590 617583     39
Elapsed time = 179.64 sec. (168111.43 ticks, tree = 34.17 MB, solutions = 8)
 630340 127265        cutoff              0.2143        0.2099  3806352    2.09%   

11/12/19 @ 02:00 PM | running sequential rounding on 6 solutions
11/12/19 @ 02:00 PM | best objective value: 0.3660
11/12/19 @ 02:00 PM | sequential rounding produced 2 integer solutions
11/12/19 @ 02:00 PM | best objective value: 0.3457
11/12/19 @ 02:00 PM | polishing 7 solutions
11/12/19 @ 02:00 PM | best objective value: 0.3457
11/12/19 @ 02:00 PM | polishing produced 5 integer solutions
11/12/19 @ 02:00 PM | initialization produced 10 feasible solutions
11/12/19 @ 02:00 PM | best objective value: 0.2190
11/12/19 @ 02:00 PM | ------------------------------------------------------------
11/12/19 @ 02:00 PM | completed initialization procedure
11/12/19 @ 02:00 PM | ------------------------------------------------------------
11/12/19 @ 02:00 PM | 253 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_Time

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:00 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2190.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:15 PM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2198.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 476940 112611        0.2108    16        0.2134        0.2081  2646046    2.51%        alpha_20 D 476940 476939     31
 487430 112591        0.2132     7        0.2134        0.2082  2703104    2.46%           rho_8 U 487430 487429     42
 499170 114788        0.2116     9        0.2134        0.2083  2757189    2.41%        alpha_17 D 499170 499169     33
 510710 115259        0.2096    16        0.2134        0.2084  2808766    2.38%           rho_9 U 510710 510709     34
 522190 117052        0.2101     2        0.2134        0.2084  2862479    2.34%        alpha_17 U 522190 275321     41
 532593 118273        cutoff              0.2134        0.2085  2919960    2.32%        alpha_18 D 532593 532591     39
Elapsed time = 170.11 sec. (168119.86 ticks, tree = 32.98 MB, solutions = 7)
 543805 120043        cutoff              0.2134        0.2085  2974596    2.29%           rho_9 D 543805  97212     20
 555357 120750        cutoff              0.2134        0.2086  3030719    2.25%   

11/12/19 @ 02:21 PM | CPA produced 2 cuts
11/12/19 @ 02:21 PM | running naive rounding on 6 solutions
11/12/19 @ 02:21 PM | best objective value: 0.3663
11/12/19 @ 02:21 PM | rounding produced 5 integer solutions
11/12/19 @ 02:21 PM | best objective value is 0.3456
11/12/19 @ 02:21 PM | running sequential rounding on 6 solutions
11/12/19 @ 02:21 PM | best objective value: 0.3663
11/12/19 @ 02:21 PM | sequential rounding produced 3 integer solutions
11/12/19 @ 02:21 PM | best objective value: 0.3456
11/12/19 @ 02:21 PM | polishing 8 solutions
11/12/19 @ 02:21 PM | best objective value: 0.3456
11/12/19 @ 02:21 PM | polishing produced 5 integer solutions
11/12/19 @ 02:21 PM | initialization produced 12 feasible solutions
11/12/19 @ 02:21 PM | best objective value: 0.2208
11/12/19 @ 02:21 PM | ------------------------------------------------------------
11/12/19 @ 02:21 PM | completed initialization procedure
11/12/19 @ 02:21 PM | -----------------------------------------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:21 PM | adding 255 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2208.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 400476 67995        cutoff              0.2144        0.2112  2153555    1.48%           rho_7 U 400476  55488     37
 409860 68011        cutoff              0.2144        0.2113  2196869    1.44%          rho_20 D 409860 364831     32
 419034 66761        cutoff              0.2144        0.2114  2240641    1.41%           rho_7 D 419034 419032     27
 428571 66244        cutoff              0.2144        0.2114  2284616    1.38%          rho_10 U 428571 428568     27
 438032 65801        cutoff              0.2144        0.2115  2330821    1.34%          rho_10 U 438032 438030     29
 447749 65066        cutoff              0.2144        0.2116  2375917    1.31%          rho_13 U 447749 430662     34
 456920 62777        0.2138     2        0.2144        0.2117  2420181    1.25%           rho_0 D 456920 456919     27
Elapsed time = 158.02 sec. (168122.66 ticks, tree = 20.54 MB, solutions = 8)
 466388 61114        0.2118    14        0.2144        0.2118  2462478    1.19%           

In [20]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_validation_score'])

(0.5856470365221786, 0.7851974964391406)

In [28]:
#### save results
summary_violent2_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}

In [29]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["Violent",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)